In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install pandas
# !pip install -U sentence-transformers
# !pip install --upgrade transformers

In [2]:
# !pip show datasets

In [3]:
# !pip uninstall torchvision
# !pip install torchvision


In [4]:
import torch
print(torch.__version__)  
print(torch.version.cuda)  
print(torch.cuda.is_available())  
import torch
print(torch.__config__.show())
print(torch.cuda.device_count())  
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")  
print(torch.backends.cudnn.enabled)  


2.6.0+cu126
12.6
True
PyTorch built with:
  - GCC 11.2
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2024.2-Product Build 20240605 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.6
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 90.5.1
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, COMMIT_SHA=2236df1770800ffea5697b11b0bb0d910b2e59e1, CUDA_VERSION=12.6, CUDNN_VERSION=9.5.1, CXX_COMPILER=/opt/rh/gcc-toolset-11/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=1 -fvi

In [5]:
import pandas as pd

df = pd.read_csv('PMCXML6.tsv', delimiter='\t')
df.head()

,fileid,filename,AccessionID,pmid,MID,title,registries,tag,isNCT,haveAlready,isReal,NCTclean,section,altsection,noLic,NLMPT,MTPT,TPscore,Country,year
0,925751,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10515754,37745325,PMC10515754,article > back > ack > p > ext-link,NCT00001360,ext-link,1,0,1,NCT00001360,Other,Other,0,preprint,NaN,NaN,USA,2024.0
1,925752,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT02770716,p,1,0,1,NCT02770716,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.999005,USA,2023.0
2,925753,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT01143246,p,1,0,1,NCT01143246,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.983015,USA,2023.0
3,925754,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT02770716,p,1,0,1,NCT02770716,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.999005,USA,2023.0
4,925755,oa_comm_xml.incr.2024-06-28.tar.gz:PMC010xxxxx...,PMC10749708,37622521,PMC10749708,article > body > INTRODUCTION > p,NCT01143246,p,1,0,1,NCT01143246,Introduction,Introduction,0,NaN,retrospective studies|multicenter study,0.983015,USA,2023.0


In [3]:
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

jaccard_similarity({'Results', 'Results'}, {'Other', 'Conclusion'})


0.0

In [7]:

x = []
sections = []
for ind, row in df.iterrows():
    parts = row['title'].split('>')
    desired_text = parts[2].strip()
    if desired_text != "":
        x.append(desired_text)
        sections.append({row['section'], row['altsection']})

x, sections
    

(['ack',
  'INTRODUCTION',
  'INTRODUCTION',
  'INTRODUCTION',
  'INTRODUCTION',
  'ack',
  'ack',
  'article-meta',
  'article-meta',
  'METHODS',
  'METHODS',
  'METHODS',
  'METHODS',
  'article-meta',
  'article-meta',
  'Materials and methods',
  'Materials and methods',
  'Materials and methods',
  'Materials and methods',
  'ref-list',
  'Methods',
  'Methods',
  'Methods',
  'Methods',
  'Unlocking the potential of MSCs and exosomes in Dermatology: a promising cell-free therapeutic approach',
  'Unlocking the potential of MSCs and exosomes in Dermatology: a promising cell-free therapeutic approach',
  'Unlocking the potential of MSCs and exosomes in Dermatology: a promising cell-free therapeutic approach',
  'Unlocking the potential of MSCs and exosomes in Dermatology: a promising cell-free therapeutic approach',
  'Unlocking the potential of MSCs and exosomes in Dermatology: a promising cell-free therapeutic approach',
  'Unlocking the potential of MSCs and exosomes in Dermato

In [8]:
import numpy as np

# 假設你的 list 是 data_list
data_list = np.arange(4177918)  # 如果是數字
# data_list = np.array(你的大數據list)  # 如果是任意內容

# 產生 100,000 組隨機配對
pairs = np.random.choice(data_list, size=(100000, 2), replace=False)

# 檢查前 5 組
print(pairs[:5])


[[1899478 1715907]
 [2388038 2796961]
 [4021796  748100]
 [2414530 2271532]
 [1900080 3859199]]


In [9]:
training_data = []
for ind1, ind2 in pairs:
    training_data.append((x[ind1], x[ind2], jaccard_similarity(sections[ind1], sections[ind2])))
training_data

[('4. Therapeutic Targeting of E3 Ligases in Cancer Immunotherapy',
  'Materials and methods',
  0.5),
 ('Application of protein fusion inhibitors in clinical practice',
  'Results',
  0.0),
 ('Clinical translation', '4. Treatment of mHLH', 1.0),
 ('Methods', 'Viral Entry Inhibitors', 0.0),
 ('HCV-RELATED HCC miRNA-BASED APPROACHES', 'article-meta', 0.0),
 ('Results', 'Clinics and pathology', 0.0),
 ('ack', 'CONCLUSION', 0.0),
 ('The effect of antidepressant therapies on neuroplasticity and neuroprotection',
  'Source of Cells for Retinal Transplantation',
  0.0),
 ('Hedgehog signaling in AML', 'Methods', 0.0),
 ('article-meta', 'Introduction', 0.0),
 ('METHODS', 'table-wrap', 0.0),
 ('Signaling Pathways of RILI and Targeted Therapies for RILI',
  'CLINICAL STUDIES OF TARGETED NEUROBLASTOMA THERAPY',
  0.3333333333333333),
 ('PB0671', 'Introduction', 0.0),
 ('New perspectives in treatment of BMs', 'notes', 0.5),
 ('Background', 'Strategies of tumor treatment', 0.0),
 ('Methods', '2. An

In [10]:
# !pip show torch transformers

In [11]:
# !pip show sentence-transformers

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses

class SentencePairDataset(Dataset):
    def __init__(self, sentence_pairs):
        self.samples = [InputExample(texts=[s1, s2], label=float(score)) for s1, s2, score in sentence_pairs]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


# 建立 Dataset
train_dataset = SentencePairDataset(training_data)

# sentence-transformers 需要一個特殊的 DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [26]:
from sentence_transformers import SentenceTransformer, losses

# 1. 載入預訓練模型
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. 使用 Cosine Similarity Loss
loss = losses.AnglELoss(model)


In [14]:
# !pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [15]:
from datasets import Dataset

In [27]:
# 訓練模型
model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=3,
    warmup_steps=100
)


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,6.196600
1000,5.769000
1500,5.429100
2000,5.361600
2500,5.502200
3000,5.373000
3500,5.309500
4000,5.240300
4500,5.248100
5000,5.218800


In [17]:
# !pip install transformers[torch]

In [28]:
# 載入模型
model.save("fine_tuned_sentence_bert_model_AnglELoss")
fine_tuned_model = SentenceTransformer("fine_tuned_sentence_bert_model_AnglELoss")

# 測試新模型
sentences = ["app-group", "Clinical trials assessing bsAbs targeting immunomodulatory checkpoints"]
embeddings = fine_tuned_model.encode(sentences)

print(embeddings.shape)  # (2, 384) -> 2 個句子，每個有 384 維度的嵌入


(2, 384)


In [29]:
embeddings

array([[ 0.04163747, -0.01812563, -0.01506701, -0.03651993, -0.00364399,
        -0.00736661,  0.05641595,  0.02694586,  0.01214726, -0.00627193,
         0.04511201, -0.05996134,  0.01921945, -0.00651108,  0.04098387,
         0.04525718, -0.05798994,  0.04601544, -0.02428723, -0.06652851,
        -0.03902736,  0.03031879,  0.09346688,  0.04533965,  0.0067825 ,
         0.04427031, -0.08998861,  0.02316422,  0.0838801 ,  0.00605553,
         0.04548264, -0.02715932, -0.02285313,  0.03773755, -0.03267668,
         0.05948149,  0.01880023, -0.11110268, -0.03138827, -0.02779789,
         0.06593427,  0.07478728, -0.04996443,  0.1065045 , -0.08972514,
        -0.03793259,  0.01137837,  0.04462018,  0.00310187,  0.06228391,
         0.1306292 , -0.01950498,  0.01533441,  0.06914211,  0.01997553,
         0.06133073, -0.02074998,  0.01316621, -0.02336246,  0.02454231,
         0.04532381,  0.00448275,  0.0292798 ,  0.03403847, -0.02719993,
         0.06238527, -0.01776674,  0.02869806,  0.0